In [14]:
import serial
import serial.tools.list_ports
import pandas as pd
import time
import os
import re
import math

def select_serial_port():
    ports = serial.tools.list_ports.comports()
    print("Port serial yang tersedia:")
    if not ports:
        print("-> Tidak ada port serial yang ditemukan. Pastikan perangkat Anda terhubung.")
        return None
    for i, port in enumerate(ports):
        print(f"  {i + 1}: {port.device} - {port.description}")
    while True:
        try:
            choice = int(input("Pilih nomor port untuk ESP32 Anda: "))
            if 1 <= choice <= len(ports):
                return ports[choice - 1].device
            else:
                print("Pilihan tidak valid.")
        except (ValueError, IndexError):
            print("Input tidak valid. Harap masukkan nomor.")

def parse_line_kasuse(line: str):
    # Expektasi kolom: Idx, Actual, "Pred (label)", Conf, Time
    parts = [p.strip() for p in line.split('\t') if p.strip() != ""]
    if len(parts) < 5:
        raise ValueError("Kolom kurang untuk Kasus E")
    # Idx
    idx = int(float(parts[0]))
    # Actual
    actual = int(float(parts[1]))
    # Pred(Label)
    m = re.match(r"^\s*(\d+)\s*\(([^)]+)\)\s*$", parts[2])
    if m:
        pred = int(m.group(1))
        pred_label = m.group(2).strip()
    else:
        # fallback: bisa jadi hanya angka atau hanya label
        # coba angka dulu
        try:
            pred = int(float(parts[2]))
            pred_label = ""
        except:
            pred = math.nan
            pred_label = parts[2]
    # Conf
    conf = float(parts[3])
    # Time (us)
    # hapus potensi suffix "us" jika ada
    time_str = parts[4].replace("us", "").strip()
    t_us = float(time_str)
    return [idx, actual, pred, pred_label, conf, t_us]

def parse_line_kasusd(line: str):
    # Expektasi kolom: Sample, Actual, Predicted, Time
    parts = [p.strip() for p in line.split('\t') if p.strip() != ""]
    if len(parts) < 4:
        raise ValueError("Kolom kurang untuk Kasus D")
    idx = int(float(parts[0]))
    actual = int(float(parts[1]))
    pred = int(float(parts[2]))
    pred_label = ""  # tidak tersedia pada format D
    conf = math.nan  # tidak tersedia pada format D
    time_str = parts[3].replace("us", "").strip()
    t_us = float(time_str)
    return [idx, actual, pred, pred_label, conf, t_us]

def main():
    port = select_serial_port()
    if not port:
        return

    baud_rate = 115200
    output_filename = input("Masukkan nama file Excel output (contoh: hasil_benchmark.xlsx): ").strip() or "hasil_benchmark.xlsx"
    if not output_filename.lower().endswith('.xlsx'):
        output_filename += '.xlsx'

    print("\n--- Konfigurasi ---")
    print(f"  Port        : {port}")
    print(f"  Baud Rate   : {baud_rate}")
    print(f"  File Output : {output_filename}")

    data_rows = []
    summary_data = {}
    is_reading_data = False
    parser = None  # fungsi parser yang dipilih berdasar header
    header_seen = ""

    try:
        with serial.Serial(port, baud_rate, timeout=10) as ser:
            print("\nKoneksi berhasil. Menunggu data...")
            print(">>> RESET ESP32 sekarang untuk memulai benchmark. <<<")

            while True:
                line_bytes = ser.readline()
                if not line_bytes:
                    print("\nTidak ada data selama 10 detik. Proses dianggap selesai.")
                    break

                line = line_bytes.decode('utf-8', errors='ignore').strip()
                if line:
                    print(f"Diterima: {line}")

                # Deteksi header (Kasus E)
                if ("Idx" in line and "Actual" in line and "Pred" in line and "Time" in line) and ("Pred(Label)" in line):
                    header_seen = line
                    is_reading_data = True
                    parser = parse_line_kasuse
                    continue

                # Deteksi header (Kasus D legacy)
                if ("Sample" in line and "Actual" in line and "Predicted" in line and "Time" in line):
                    header_seen = line
                    is_reading_data = True
                    parser = parse_line_kasusd
                    continue

                # Penanda akhir tabel
                if "===" in line:
                    is_reading_data = False
                    continue

                # Baca baris data
                if is_reading_data and parser is not None:
                    try:
                        row = parser(line)
                        data_rows.append(row)
                    except Exception as e:
                        # lewati baris non-data (mis. judul banner)
                        pass

                # Ringkasan
                if "Akurasi Test:" in line:
                    try:
                        accuracy_value = float(line.split(':')[1].strip().split(' ')[0])
                        summary_data["Metrik"] = ["Akurasi Test (%)", "Rata-rata Waktu Inferensi (us)"]
                        summary_data["Nilai"] = [accuracy_value]
                    except:
                        pass

                if "Rata-rata waktu inferensi:" in line:
                    try:
                        time_value_str = line.split(':')[1].strip().split(' ')[0]
                        time_value = float(time_value_str)
                        if "Nilai" in summary_data:
                            summary_data["Nilai"].append(time_value)
                        time.sleep(0.5)
                        break
                    except:
                        pass

    except serial.SerialException as e:
        print(f"\nERROR: Tidak dapat membuka port {port}. {e}")
        return
    except Exception as e:
        print(f"\nTerjadi kesalahan tak terduga: {e}")
        return

    if not data_rows:
        print("\nTidak ada data valid yang ditangkap. File Excel tidak akan dibuat.")
        return

    print("\nPenangkapan data selesai. Menyiapkan file Excel...")

    try:
        # Header konsisten untuk output
        columns = ["Idx", "Actual", "Pred", "Pred_Label", "Conf", "Time (us)"]
        df_data = pd.DataFrame(data_rows, columns=columns)
        df_summary = pd.DataFrame(summary_data) if summary_data else pd.DataFrame({"Metrik": [], "Nilai": []})

        with pd.ExcelWriter(output_filename, engine='openpyxl') as writer:
            df_data.to_excel(writer, sheet_name='Hasil Benchmark', index=False)
            start_row_summary = len(df_data) + 3
            if not df_summary.empty:
                df_summary.to_excel(writer, sheet_name='Hasil Benchmark', index=False,
                                    startrow=start_row_summary, header=True)

        print(f"\nBERHASIL! Data telah disimpan ke '{os.path.abspath(output_filename)}'")
        if header_seen:
            print("\nHeader terdeteksi:", header_seen)

    except Exception as e:
        print(f"\nERROR: Gagal menulis ke file Excel: {e}")

if __name__ == "__main__":
    main()

Port serial yang tersedia:
  1: COM3 - USB Serial Device (COM3)
  2: COM7 - Standard Serial over Bluetooth link (COM7)
  3: COM6 - Standard Serial over Bluetooth link (COM6)

--- Konfigurasi ---
  Port        : COM3
  Baud Rate   : 115200
  File Output : E_4_I_RP2350.xlsx

Koneksi berhasil. Menunggu data...
>>> RESET ESP32 sekarang untuk memulai benchmark. <<<
Diterima: --- TFLite INT8 Model Benchmark (Case E, RP2350/RP2040) ---
Diterima: Starting benchmark for INT8 model (Case E)...
Diterima: --------------------------------------------------
Diterima: Sample	Actual	Predicted	Time (us)
Diterima: --------------------------------------------------
Diterima: 0	1	1	7372
Diterima: 1	0	0	7189
Diterima: 2	0	0	7172
Diterima: 3	1	1	7305
Diterima: 4	2	2	7237
Diterima: 5	2	2	7212
Diterima: 6	2	2	7203
Diterima: 7	2	2	7178
Diterima: 8	1	1	7184
Diterima: 9	1	1	7271
Diterima: 10	2	2	7247
Diterima: 11	0	0	7168
Diterima: 12	1	1	7228
Diterima: 13	2	2	7200
Diterima: 14	1	1	7182
Diterima: 15	0	0	7233
Dit

In [15]:
import os
import re
import pandas as pd

# === Konfigurasi ===
ROOT_DIR = "."   # ganti ke folder berkas .xlsx jika perlu
SHEET = "Hasil Benchmark"
OUT_FILE = "E_summary.xlsx"

# Pola nama: E_<modelIdx>_<F|I>_<ESP32|RP2350>.xlsx
NAME_RE = re.compile(r"^E_(\d+)_([FI])_(ESP32|RP2350)\.xlsx$", re.IGNORECASE)

def parse_filename(fname: str):
    m = NAME_RE.match(fname)
    if not m:
        return None
    model_idx = int(m.group(1))
    quant = "Float32" if m.group(2).upper() == "F" else "INT8"
    platform = m.group(3).upper()
    return model_idx, quant, platform

def read_summary_from_sheet(path: str):
    """
    Coba ambil 'Akurasi Test (%)' & 'Rata-rata Waktu Inferensi (us)' dari blok ringkasan.
    Jika tidak ada, kembalikan (None, None) agar dihitung ulang dari data.
    """
    try:
        df = pd.read_excel(path, sheet_name=SHEET)
    except Exception:
        return None, None, None

    # Cek apakah ada kolom 'Metrik' dan 'Nilai'
    if {"Metrik", "Nilai"}.issubset(df.columns):
        # Ambil baris yang relevan
        a = df.loc[df["Metrik"].astype(str).str.contains("Akurasi", case=False, na=False), "Nilai"]
        t = df.loc[df["Metrik"].astype(str).str.contains("Rata-rata Waktu", case=False, na=False), "Nilai"]
        acc = float(a.iloc[0]) if not a.empty else None
        avg = float(t.iloc[0]) if not t.empty else None
        return acc, avg, df

    # Jika tidak ada ringkasan, tetap kembalikan df agar bisa dihitung manual
    return None, None, df

def compute_from_data(df: pd.DataFrame):
    """
    Hitung akurasi & rata-rata waktu dari tabel data utama.
    Kompatibel dengan dua format:
    - Kasus D: columns ~ ['Sample','Actual','Predicted','Time (us)']
    - Kasus E: columns ~ ['Idx','Actual','Pred(Label)','Conf','Time (us)'] -> kita butuh kolom Pred saja.
    """
    cols = {c.lower(): c for c in df.columns}  # case-insensitive map

    # deteksi kolom waktu
    time_col = cols.get("time (us)") or cols.get("time") or None

    # deteksi prediksi
    pred_col = cols.get("predicted") or cols.get("pred") or cols.get("pred(label)") or None
    actual_col = cols.get("actual") or None

    if actual_col is None or pred_col is None or time_col is None:
        return None, None

    # Jika 'Pred(Label)' berbentuk "2 (walk)", ambil angka depannya
    pred_series = df[pred_col].astype(str).str.extract(r"(-?\d+)")[0].astype(float)
    act_series = pd.to_numeric(df[actual_col], errors="coerce")

    time_series = pd.to_numeric(df[time_col], errors="coerce")

    valid_mask = act_series.notna() & pred_series.notna()
    if valid_mask.sum() == 0:
        return None, None

    acc = (pred_series[valid_mask] == act_series[valid_mask]).mean() * 100.0
    avg = time_series.dropna().mean() if time_series.notna().any() else None
    return float(acc) if acc is not None else None, float(avg) if avg is not None else None

def main():
    rows = []
    for fname in os.listdir(ROOT_DIR):
        if not fname.lower().endswith(".xlsx"):
            continue
        meta = parse_filename(fname)
        if not meta:
            continue

        model_idx, quant, platform = meta
        path = os.path.join(ROOT_DIR, fname)

        acc, avg, df = read_summary_from_sheet(path)
        if acc is None or avg is None:
            # Hitung manual dari data (ambil blok tabel teratas sebelum ringkasan)
            # Jika file berisi dua blok pada sheet yang sama, baris 'Metrik' akan ada; ambil hanya baris non-'Metrik'
            try:
                if df is None:
                    df = pd.read_excel(path, sheet_name=SHEET)
                df_data = df.copy()
                if "Metrik" in df_data.columns:
                    # potong sebelum baris ringkasan (baris pertama yang punya 'Metrik')
                    first_metric_idx = df_data.index[df_data["Metrik"].notna()].min()
                    df_data = df_data.iloc[:first_metric_idx] if pd.notna(first_metric_idx) else df_data
                # buang kolom kosong penuh
                df_data = df_data.dropna(axis=1, how="all")
                acc2, avg2 = compute_from_data(df_data)
                acc = acc if acc is not None else acc2
                avg = avg if avg is not None else avg2
            except Exception:
                pass

        rows.append({
            "File": fname,
            "Case": "E",
            "Model_Idx": model_idx,
            "Quant": quant,              # Float32 / INT8
            "Platform": platform,        # ESP32 / RP2350
            "Akurasi Test (%)": acc,
            "Avg Inference (us)": avg
        })

    if not rows:
        print("Tidak ada file yang sesuai pola ditemukan.")
        return

    out = pd.DataFrame(rows).sort_values(["Model_Idx", "Quant", "Platform"])
    # kolom rapi
    out = out[["Case","Model_Idx","Quant","Platform","Akurasi Test (%)","Avg Inference (us)","File"]]
    print(out.to_string(index=False))

    with pd.ExcelWriter(OUT_FILE, engine="openpyxl") as w:
        out.to_excel(w, sheet_name="Summary", index=False)

    print(f"\n✅ Ringkasan disimpan ke '{OUT_FILE}'")

if __name__ == "__main__":
    main()


Case  Model_Idx   Quant Platform  Akurasi Test (%)  Avg Inference (us)              File
   E          1 Float32    ESP32         97.777778          822.944444  E_1_F_ESP32.xlsx
   E          1 Float32   RP2350         97.777778         1292.066667 E_1_F_RP2350.xlsx
   E          1    INT8    ESP32         97.777778          675.755556  E_1_I_ESP32.xlsx
   E          1    INT8   RP2350         97.777778          509.211111 E_1_I_RP2350.xlsx
   E          2 Float32    ESP32         97.777778         2829.222222  E_2_F_ESP32.xlsx
   E          2 Float32   RP2350         97.777778         2640.522222 E_2_F_RP2350.xlsx
   E          2    INT8    ESP32         97.777778         1218.911111  E_2_I_ESP32.xlsx
   E          2    INT8   RP2350         97.777778          996.788889 E_2_I_RP2350.xlsx
   E          3 Float32    ESP32         98.888889         6891.055556  E_3_F_ESP32.xlsx
   E          3 Float32   RP2350         98.888889         5423.133333 E_3_F_RP2350.xlsx
   E          3    IN